<a href="https://colab.research.google.com/github/Coltcult/fantastic-computing-machine/blob/main/Python_Touch_Graphics_Game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pygame
import random

# Initialize Pygame
pygame.init()

# Set up the screen
screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("Touchy Survival Game")

# Define colors
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
GRAY = (100, 100, 100)

# Load assets (simple shapes for demonstration)
player_size = 50
player_x = screen_width / 2 - player_size / 2
player_y = screen_height / 2 - player_size / 2
player_speed = 5

rock_size = 30
rocks = []
num_rocks = 5
for _ in range(num_rocks):
    rock_x = random.randint(0, screen_width - rock_size)
    rock_y = random.randint(0, screen_height - rock_size)
    rocks.append((rock_x, rock_y))

tree_size = 40
trees = []
num_trees = 8
for _ in range(num_trees):
    tree_x = random.randint(0, screen_width - tree_size)
    tree_y = random.randint(0, screen_height - tree_size)
    trees.append((tree_x, tree_y))

food_size = 20
foods = []
num_foods = 3
for _ in range(num_foods):
    food_x = random.randint(0, screen_width - food_size)
    food_y = random.randint(0, screen_height - food_size)
    foods.append((food_x, food_y))

water_size = 20
waters = []
num_waters = 3
for _ in range(num_waters):
    water_x = random.randint(0, screen_width - water_size)
    water_y = random.randint(0, screen_height - water_size)
    waters.append((water_x, water_y))

# Game state variables
hunger = 100
thirst = 100
health = 100
inventory = {'wood': 0, 'rock': 0, 'food': 0, 'water': 0}
game_over = False
day = 1
time_of_day = 6  # 6:00
shelter_built = False

# Function to draw game objects
def draw_player():
    pygame.draw.rect(screen, BLUE, (player_x, player_y, player_size, player_size))

def draw_rocks():
    for rock_x, rock_y in rocks:
        pygame.draw.rect(screen, GRAY, (rock_x, rock_y, rock_size, rock_size))

def draw_trees():
    for tree_x, tree_y in trees:
        pygame.draw.rect(screen, GREEN, (tree_x, tree_y, tree_size, tree_size))

def draw_foods():
    for food_x, food_y in foods:
        pygame.draw.circle(screen, RED, (int(food_x + food_size / 2), int(food_y + food_size / 2)), food_size // 2)

def draw_waters():
    for water_x, water_y in waters:
        pygame.draw.circle(screen, BLUE, (int(water_x + water_size / 2), int(water_y + water_size / 2)), water_size // 2)

def draw_hud():
    font = pygame.font.Font(None, 30)
    hunger_text = font.render(f"Hunger: {hunger}", True, WHITE)
    thirst_text = font.render(f"Thirst: {thirst}", True, WHITE)
    health_text = font.render(f"Health: {health}", True, WHITE)
    wood_text = font.render(f"Wood: {inventory['wood']}", True, WHITE)
    rock_text = font.render(f"Rock: {inventory['rock']}", True, WHITE)
    food_text = font.render(f"Food: {inventory['food']}", True, WHITE)
    water_text = font.render(f"Water: {inventory['water']}", True, WHITE)
    day_text = font.render(f"Day: {day}", True, WHITE)
    time_text = font.render(f"Time: {time_of_day:02d}:00", True, WHITE)

    screen.blit(hunger_text, (10, 10))
    screen.blit(thirst_text, (10, 40))
    screen.blit(health_text, (10, 70))
    screen.blit(wood_text, (10, 100))
    screen.blit(rock_text, (10, 130))
    screen.blit(food_text, (10, 160))
    screen.blit(water_text, (10, 190))
    screen.blit(day_text, (screen_width - 100, 10))
    screen.blit(time_text, (screen_width - 100, 40))

    if game_over:
        font = pygame.font.Font(None, 60)
        game_over_text = font.render("Game Over", True, RED)
        screen.blit(game_over_text, (screen_width / 2 - 100, screen_height / 2 - 30))

# Function to handle player movement
def move_player(touch_x, touch_y):
    global player_x, player_y
    # Calculate the direction vector
    dx = touch_x - (player_x + player_size / 2)
    dy = touch_y - (player_y + player_size / 2)

    # Normalize the direction vector
    distance = (dx ** 2 + dy ** 2) ** 0.5
    if distance > 0:
        dx /= distance
        dy /= distance

    # Move the player
    player_x += dx * player_speed
    player_y += dy * player_speed

    # Keep player within bounds
    player_x = max(0, min(player_x, screen_width - player_size))
    player_y = max(0, min(player_y, screen_height - player_size))

# Function to handle interactions
def handle_interactions():
    global player_x, player_y, hunger, thirst, health, inventory, rocks, trees, foods, waters, game_over, time_of_day, day, shelter_built

    # Rock interaction
    for i, (rock_x, rock_y) in enumerate(rocks):
        if (
            player_x < rock_x + rock_size
            and player_x + player_size > rock_x
            and player_y < rock_y + rock_size
            and player_y + player_size > rock_y
        ):
            inventory['rock'] += 1
            rocks.pop(i)
            print("Picked up a rock!")
            break  # Only pick up one rock per frame

    # Tree interaction
    for i, (tree_x, tree_y) in enumerate(trees):
        if (
            player_x < tree_x + tree_size
            and player_x + player_size > tree_x
            and player_y < tree_y + tree_size
            and player_y + player_size > tree_y
        ):
            inventory['wood'] += 1
            trees.pop(i)
            print("Picked up wood!")
            break

    # Food interaction
    for i, (food_x, food_y) in enumerate(foods):
        if (
            player_x < food_x + food_size
            and player_x + player_size > food_x
            and player_y < food_y + food_size
            and player_y + player_size > food_y
        ):
            inventory['food'] += 1
            foods.pop(i)
            hunger = max(0, hunger + 20)  #restore hunger
            print("Picked up food!")
            break

    # Water interaction
    for i, (water_x, water_y) in enumerate(waters):
        if (
            player_x < water_x + water_size
            and player_x + player_size > water_x
            and player_y < water_y + water_size
            and player_y + player_size > water_y
        ):
            inventory['water'] += 1
            waters.pop(i)
            thirst = max(0, thirst + 30)  #restore thirst
            print("Picked up water!")
            break

    # Build shelter
    if inventory['wood'] >= 10 and inventory['rock'] >= 5 and not shelter_built:
        shelter_built = True
        inventory['wood'] -= 10
        inventory['rock'] -= 5
        print("You built a shelter")

def update_game_state():
    global hunger, thirst, health, game_over, time_of_day, day

    hunger -= 1
    thirst -= 2
    time_of_day += 1

    if time_of_day >= 24:
        time_of_day = 0
        day += 1

    if not shelter_built and (time_of_day >= 18 or time_of_day < 6): #night
        health -= 5
        print("Its night and you dont have a shelter")

    if hunger <= 0:
        health -= 10
        print("You are starving")
    if thirst <= 0:
        health -= 15
        print("You are dehydrated")

    if health <= 0:
        game_over = True
        print("You died. Game Over")

# Main game loop
running = True
while running:
    screen.fill((0, 128, 0))  # Green background

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            touch_x, touch_y = event.pos
            move_player(touch_x, touch_y)

    draw_rocks()
    draw_trees()
    draw_foods()
    draw_waters()
    draw_player()
    draw_hud()

    if not game_over:
        handle_interactions()
        update_game_state()

    pygame.display.flip()
    pygame.time.delay(16)  # Limit frame rate to ~60 FPS

pygame.quit()

pygame 2.6.1 (SDL 2.28.4, Python 3.11.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
Its night and you dont have a shelter
Its night and you dont have a shelter
Its night and you dont have a shelter
Its night and you dont have a shelter
Its night and you dont have a shelter
Its night and you dont have a shelter
Its night and you dont have a shelter
Its night and you dont have a shelter
Its night and you dont have a shelter
Its night and you dont have a shelter
Its night and you dont have a shelter
Its night and you dont have a shelter
Its night and you dont have a shelter
Its night and you dont have a shelter
Its night and you dont have a shelter
Its night and you dont have a shelter
Its night and you dont have a shelter
Its night and you dont have a shelter
Its night and you dont have a shelter
Its night and you dont have a shelter
You died. Game Over
